# 1. `Network`: the base object of scikit-rf
In this first part of this short course, we give an overview of the microwave network analysis 
features of scikit-rf. For the rest of the short course, it is  assumed that scikit-rf (**skrf**) has been imported as `rf`:


In [ ]:
import skrf as rf

# Creating Networks

<img src="figures/Nport_network.png" align="right">

**skrf** provides a Python object for a N-port microwave `Network`. A `Network` can be created in a number of ways:
 - from a Touchstone file (.sNP files)
 - from S-parameters
 - from Z-parameters
 - from other RF parameters (Y, ABCD, T, etc.) 
 
Some examples for each situation are given after.

## Creating Network from Touchstone file
[Touchstone file](https://en.wikipedia.org/wiki/Touchstone_file) (`.sNp` files, with `N` being the number of ports) is a _de facto_ standard to export N-port network parameter data and noise data of linear active devices, passive filters, passive devices, or interconnect networks. Creating a Network from a Touchstone file is simple:

In [ ]:
ring_slot = rf.Network('data/ring slot.s2p')

Note that some softwares, such as ANSYS HFSS, add additional information to the Touchstone standard, such as comments, simulation parameters, Port Impedance or Gamma (wavenumber). These data are also imported if detected. 

	
A short description of the network will be printed out if entered onto the command line
	

In [ ]:
ring_slot  # or: print(ring_slot)

### Diving into the content of the `Network` object 

The `Network` object stores the scattering parameters of a the Network, but also many other informations:
- the frequency points 
- the characteristic impedances of the ports (which can be frequency dependent)

The S-parameter of a N-port network is stored in a `Network` as a Numpy array of shape `(nb_f, N, N)`, where `nb_f` is the number of frequency points and `N` the number of ports of the network. 

The S-parameter array is accessible with the `.s` parameter of the Network. For instance, our previous example is a 2-port with 201 frequency points, so the S-parameters array is of shape (201,2,2):

In [ ]:
ring_slot.s.shape

### Creating Network from s-parameters
Networks can also be created by directly by passing the values for the `frequency` points and tehe s-parameters (and optionally the port impedance `z0`).

In [ ]:
import numpy as np 
# dummy 2-port network from Frequency and s-parameters
freq = rf.Frequency(start=1, stop=10, npoints=101, unit='ghz')

s = np.random.rand(101, 2, 2) + 1j*np.random.rand(101, 2, 2)  # random complex numbers 
# if not passed, will assume z0=50. name is optional but it's a good practice.
ntwk = rf.Network(frequency=freq, s=s, name='random values 2-port') 
print(ntwk)

# Exercice: creating a Network from S-parameters
Often, s-parameters are stored in separate arrays, say Numpy array `S11`, `S21`, `S12` and `S22`. 

In [ ]:
# let's assume we have separate arrays for the frequency and s-parameters
f = np.array([1, 2, 3, 4]) # in GHz
S11 = np.random.rand(4)
S12 = np.random.rand(4)
S21 = np.random.rand(4)
S22 = np.random.rand(4)

In such case, one needs to forge the `frequency` and the `s` array of expected shape (nbf,2,2):

In [ ]:
freq2 = rf.Frequency.from_f(f, unit='GHz')

In [ ]:
# forging S-matrix as shape (nb_f, 2, 2)
s = np.zeros((len(f), 2, 2), dtype=complex)
s[:,0,0] = S11
s[:,0,1] = S12
s[:,1,0] = S21
s[:,1,1] = S22

In [ ]:
# constructing Network object
ntw = rf.Network(frequency=freq2, s=s)
print(ntw)

If necessary, the characteristic impedance can be passed as a scalar (same for all frequencies), as a list or an array:

In [ ]:
ntw2 = rf.Network(frequency=freq, s=s, z0=25, name='same z0 for all ports')
print(ntw2)

In [ ]:
ntw3 = rf.Network(frequency=freq, s=s, z0=[20, 30], name='different z0 for each port')
print(ntw3)

In [ ]:
ntw4 = rf.Network(frequency=freq, s=s, z0=np.random.rand(101,2), 
                  name='different z0 for each frequencies and ports')
print(ntw4)

### Creating a Network from z-parameters 
As networks are also defined from their Z-parameters, there is `from_z()` method of the Network:

In [ ]:
# 1-port network example
z = 10j
Z = np.full((len(freq), 1, 1), z)  # replicate z for all frequencies

ntw = rf.Network()
ntw = ntw.from_z(Z)
ntw.frequency = freq
print(ntw)

### From other network parameters (Z, Y, ABCD, T)
It is also possible to generate Networks from other kind of RF parameters, using the conversion functions: `z2s`, `y2s`, `a2s`, `t2s`, `h2s`.

For example, the [ABCD parameters](https://en.wikipedia.org/wiki/Two-port_network#ABCD-parameters) of a serie-impedance is:
$$
\left[
\begin{array}{cc}
1 & Z \\
0 & 1
\end{array}
\right]
$$

In [ ]:
z = 20
abcd = np.array([[1, z],
                 [0, 1]])

s = rf.a2s(np.tile(abcd, (len(freq),1,1))) # (2,2) -> (N,2,2)
ntw = rf.Network(frequency=freq, s=s)
print(ntw)

## Gettint the Basic Properties of a Network
	
The basic attributes of a microwave Network are provided by the following properties :

* `Network.s` : Scattering Parameter matrix. 
* `Network.z0`  : Port Characteristic Impedance matrix.
* `Network.frequency`  : Frequency Object. 



The `Network` object has numerous other properties and methods. If you are using IPython/Jupyter, then these properties and methods can be 'tabbed' out on the command line:

In [ ]:
ring_slot # press .<TAB>


All of the network parameters are represented internally as complex `numpy.ndarray`. The s-parameters are of shape (nfreq, nport, nport)

In [ ]:
ring_slot.s.shape

## Projections
Network parameters, such as S-parameters, are complex numbers. However, it is more convenient to represent these network parameters using some mathematical projections, such as the magnitude in dB or the phase in degree.

Several mathematical projections of the network parameters are available for each network parameters. For example, if you want the magnitude in dB of the S-parameters:

In [ ]:
ring_slot.s_db

Or the phase in degrees:

In [ ]:
ring_slot.s_deg

Several other projections are available, please refer to the documentation for the complete list.

# Slicing

You can slice the network parameters attribute, such as `Network.s`, any way you want:

In [ ]:
ring_slot.s[:11,1,0]  # get first 10 values of S21

Slicing by frequency can also be done directly on Network objects like so 

In [ ]:
ring_slot[0:10] #  Network for the first 10 frequency points

or  with a human friendly string,

In [ ]:
ring_slot['80-90ghz']

Notice that slicing directly on a Network **returns a Network**:

A nice way to express slicing in both dimensions is 

In [ ]:
ring_slot.s11['80-90ghz'] 

## Other Parameters	

This tutorial focuses on s-parameters, but other network representations are available as well. Impedance and Admittance Parameters can be accessed through the parameters `Network.z` and `Network.y`, respectively. Scalar components of complex parameters, such as  `Network.z_re`, `Network.z_im` and plotting methods are available as well.

Other parameters are only available for 2-port networks, such as wave cascading parameters (`Network.t`), and  ABCD-parameters (`Network.a`)

In [ ]:
ring_slot.z[:3,...]

In [ ]:
ring_slot.plot_z_im(m=1,n=0)  

# Plotting 

Amongst other things, the methods of the Network class provide convenient ways to plot components of the network parameters, for example:

* `Network.plot_s_db` : plot magnitude of s-parameters in log scale
* `Network.plot_s_deg` : plot phase of s-parameters in degrees
* `Network.plot_s_smith` : plot complex s-parameters on Smith Chart
* ... many other are available

To plot all the four S-parameters in dB:

In [ ]:
ring_slot.plot_s_db()

	
To plot all four s-parameters of the `ring_slot` on the Smith Chart.

In [ ]:
ring_slot.plot_s_smith()

Combining this with the slicing features, 

In [ ]:
ring_slot.s11.plot_s_db(label='Full Band Response')
ring_slot.s11['82-90ghz'].plot_s_db(lw=3, label='Band of Interest')

# Arithmetic Operations 
Element-wise mathematical operations on the scattering parameter matrices are accessible through overloaded operators: `+`,`-`,`*`,`/`.

To illustrate their usage, load a couple of Networks stored in the `data` module of the package:

In [ ]:
from skrf.data import wr2p2_short as short 
from skrf.data import wr2p2_delayshort as delayshort 

In [ ]:
short - delayshort

In [ ]:
short + delayshort

In [ ]:
short * delayshort

In [ ]:
short / delayshort

All of these operations return Network types.

For example, to plot the complex difference  between  `short` and `delay_short`:

In [ ]:
difference = (short - delayshort)
difference.plot_s_mag(label='Mag of difference')

Another common application is calculating the phase difference using the division operator,

In [ ]:
(delayshort/short).plot_s_deg(label='Detrended Phase')

Linear operators can also be used with scalars or an `numpy.ndarray` that is the same length as the Network:

In [ ]:
hopen = short * (-1)
hopen.s[:3,...]

In [ ]:
rando =  hopen * np.random.rand(len(hopen))
rando.s[:3,...]

**warning** : Note that if you multiply a Network by an `numpy.ndarray`  be sure to place the array on the right side.

## Comparison of Network
Comparison operators also work with networks:

In [ ]:
short == delayshort

In [ ]:
short != delayshort

# Cascading and De-embedding
Cascading and de-embeding 2-port Networks can also be done through operators. The `cascade` function can be called through the Python power operator, `**`. To calculate a new network which is the cascaded connection of the two individual Networks `line` and `short`: 

In [ ]:
short = rf.data.wr2p2_short
line = rf.data.wr2p2_line
delayshort = line ** short

<img src="figures/cascading_delayshort.png"/>

De-embedding  can be accomplished by cascading the *inverse* of a network. The inverse of a network is accessed through the property `Network.inv`. To de-embed the `short` from `delay_short`,

In [ ]:
short_2 = line.inv ** delayshort

<img src="figures/deembedding_delayshort.png"/>

In [ ]:
short_2 == short

The cascading operator also works for to 2N-port Networks, also known as *balanced Networks*. For example, assuming you want to cascade two 4-port Network `ntw1`, `ntw2`, you can use:

`resulting_ntw = ntw1 ** ntw2`

Note that the port scheme assumed by the `**` cascading operator with 2N-port networks is:
<img src="figures/cascade_2Nport_2Nport.png"/>

## Connecting Multi-ports 

**skrf** supports the connection of arbitrary ports of N-port networks. It accomplishes this using an algorithm called *sub-network growth*,  available through the function `connect()`. 

As an example, terminating one port of an ideal 3-way splitter can be done like so,

In [ ]:
tee = rf.data.tee
tee

To connect port `1` of the tee, to port `0` of the delay short,

In [ ]:
terminated_tee = rf.connect(tee, 1, delayshort, 0)
terminated_tee

Note that this function takes into account port impedances. If two connected ports have different port impedances,  an appropriate impedance mismatch is inserted. 

For advanced connections between many arbitrary N-port Networks, the `Circuit` object is more relevant since it allows defining explicitly the connections between ports and Networks. We will see the `Circuit` class later in this short course.

	
# Interpolation and Concatenation

A common need is to change the number of frequency points of a [Network](../api/network.rst). To use the operators and cascading functions the networks involved must have matching frequencies, for instance. If two networks have different frequency information, then an error will be raised, 

In [ ]:
from skrf.data import wr2p2_line1 as line1

line1

In [ ]:
line1+line  # will fail

This problem can be solved by interpolating one of Networks along the frequency axis using `Network.resample`. 

In [ ]:
line1.resample(201)
line1

And now we can do things

In [ ]:
line1 + line

You can also interpolate from a `Frequency` object. For example, 

In [ ]:
line.interpolate_from_f(line1.frequency)

A related application is the need to combine Networks which cover different frequency ranges. Two  Networks can be concatenated (aka *stitched*) together using `stitch`, which  concatenates networks along their frequency axis. To combine a WR-2.2 Network with a WR-1.5 Network, 

In [ ]:
from skrf.data import wr2p2_line, wr1p5_line
print(wr2p2_line)
print(wr1p5_line)

In [ ]:
big_line = rf.stitch(wr2p2_line, wr1p5_line)
big_line

# Reading and Writing 
**skrf** supports writing [touchstone file format](http://en.wikipedia.org/wiki/Touchstone_file). While reading is accomplished with the Network initializer as shown before, writing is accomplished the method  `Network.write_touchstone()`.

In [ ]:
ring_slot.write_touchstone('test.s2p')